# Data Cleaning de la base scrappeada de HowLongToBeat.com

In [1]:
# Librerias necesarias
import pandas as pd
import numpy as np

## Arreglos realizados en Excel

 - Se separo el archivo en las columnas correspondientes
 - En las columnas con informacion de tiempos de completitud se modifico los casos en donde figuran horas fraccionarias (1/2 hours) por 0,5.
 - Por otro lado, en las mismas columnas, donde figuraba el tiempo en minutos, se transformo en su equivalente en horas (en formato float)
 - Los campos incompletos (en donde el scrapper no encontro informacion) se modificaron de ***"--"*** a nulos.
 - En los campos sin Rating (NR) se modificaron a nulos.
 - Se eliminaron los juegos con Type "DLC", "Expansion", etc. De esta manera solo con quedamos con jueos completos.


In [2]:
# Importo el archivo con la base de datos
df = pd.read_excel('C:/Users/Ramiro/Desktop/Digital House/Desafios/HLTB Scrapper/HowLongToBeat DB.xlsx')
#df = pd.read_excel('C:/Users/Ramiro/Desktop/Digital House/Desafios/HLTB Scrapper/HowLongToBeat DB_modificada.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48157 entries, 0 to 48156
Columns: 207 entries, Id to Updated
dtypes: float64(5), int64(195), object(7)
memory usage: 76.1+ MB


In [4]:
#Analizamos como esta compuesto el archivo
df.sample(10)

,Id,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developers,Publishers,Android,...,gen_VIRTUAL PET,gen_VIRTUAL REALITY,gen_VISUAL NOVEL,gen_WARGAME,gen_WESTERN,gen_WRESTLING,NA,EU,JP,Updated
41109,52334,The Labyrinth,NaN,NaN,NaN,NaN,NaN,Alexander Spillman,Alexander Spillman,0,...,0,0,0,0,0,0,"June 21, 2017","June 21, 2017",NaN,3 Weeks Ago
14413,17376,Mnemonic Prototype,0.416667,NaN,0.616667,0.516667,NaN,Double Fine,NaN,0,...,0,0,0,0,0,0,NaN,NaN,NaN,1.5 Weeks Ago
47363,59142,Venom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,NaN,NaN,NaN,4 Months Ago
1521,1714,CID The Dummy,NaN,6.000000,NaN,6.000000,40.0,"Oxygen Games, Twelve Games",Oxygen Games,0,...,0,0,0,0,0,0,"July 07, 2009","April 17, 2009",NaN,1.5 Months Ago
44405,55952,Edges 2,NaN,NaN,NaN,NaN,40.0,NaN,NaN,0,...,0,0,0,0,0,0,NaN,NaN,NaN,6.5 Months Ago
27070,34004,Tokyo Xtreme Racer: Drift 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,NaN,NaN,NaN,2 Years Ago
935,1069,Bird Assassin,0.816667,0.966667,1.000000,0.983333,65.0,Social Loner Studios,NaN,0,...,0,0,0,0,0,0,"February 08, 2011",NaN,NaN,4 Days Ago
24507,30670,Trog,NaN,3.000000,NaN,3.000000,65.0,"Midway, Visual Concepts","Acclaim, Software Creations",0,...,0,0,0,0,0,0,1990,NaN,NaN,1.5 Years Ago
34105,43869,Picturesque VR,NaN,NaN,NaN,NaN,NaN,Beijing Omnicity Technology,Beijing Omnicity Technology,0,...,0,1,0,0,0,0,"December 20, 2016",NaN,NaN,3 Months Ago
1688,1894,Conker's Bad Fur Day,10.500000,12.000000,12.500000,11.000000,83.0,Rare,"Rare, THQ",0,...,0,0,0,0,0,0,"March 05, 2001","April 06, 2001",NaN,1 Day Ago


El dataset esta compuesto por 48.157 observaciones con los siguientes campos:
 - __Id__: Id en la web de How Long To Beat
 - __Titulo__: Nombre del videojuego
 - __Main Story__: El tiempo necesario para completar la historia principal
 - __Main Story + Extras__: El tiempo necesario para completar la historia principal y los "sidequests"
 - __Completionist__: El tiempo necesario para completar el juego al 100%
 - __All Styles__: Un tiempo aproximado de los distintos estilos combinados
 - __Rating__: Score otorgado al juego
 - __Developer__: Lista de Desarrolladores
 - __Publishers__: Lista de compañias que publicaron el juego
 - __Playable On__: Consolas en las cuales esta disponible el juego
 - __Genres__: Generos a los cuales pertenece el juego
 - __NA__: Fecha de lanzamiento en Norteamerica
 - __EU__: Fecha de lanzamiento en Europa
 - __JP__: Fecha de lanzamiento en Japon
 - __Updated__: Ultima actualizacion de la informacion del juego

Como era de esperarse, del scrapping inicial hace falta hacer una limpieza para poder dejar listos los datos para el modelado.
Encontramos los siguientes problemas:
 - Las columnas de tiempos queremos redondearlas a dos decimales.
 - Queremos que el indice del df sea el numero de id del juego en la web para poder hacer un rastreo del mismo
 - Queremos separar las columnas de Genres, Playable On, Developers y Publishers en dummies para el modelo
 - Las columnas de Developers, Publishers, Playable On y Genres queremos que esten todas en mayusculas, para que a la hora de separarlos, no haya diferencia entre uppercase o lowercase


In [3]:
#Arreglamos las columnas con ceros extra
df['Main Story'] = df['Main Story'].astype(float).round(2)
df['Main + Extras'] = df['Main + Extras'].astype(float).round(2)
df['Completionist'] = df['Completionist'].astype(float).round(2)
df['All Styles'] = df['All Styles'].astype(float).round(2)

In [4]:
#Pasamos las columnas a UPPER
df['Genres'] = df['Genres'].str.upper()
df['Developers'] = df['Developers'].str.upper()
df['Publishers'] = df['Publishers'].str.upper()
df['Playable On'] = df['Playable On'].str.upper()

In [5]:
#Seteamos el indice 
df = df.set_index(df['Id'])
df = df.drop(columns='Id')

In [9]:
df.head(10)

,Id,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developers,Publishers,Playable On,Genres,NA,EU,JP,Updated
0,1,688(I) Hunter/Killer,9.5,24.0,50.5,15.5,NaN,SONALYSTS,ELECTRONIC ARTS,PC,SIMULATION,"July 04, 1997",NaN,NaN,1 Month Ago
1,2,Beyond Good & Evil 2,NaN,NaN,NaN,NaN,NaN,UBISOFT MONTPELLIER,UBISOFT,NaN,"ACTION, ADVENTURE",2016,NaN,NaN,3.5 Months Ago
2,10,Ico and Shadow of the Colossus Collection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
3,16,Shadow of the Comet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
4,18,'Splosion Man,8.0,10.0,18.0,9.5,76.0,TWISTED PIXEL GAMES,MICROSOFT GAMES STUDIOS,XBOX 360,"ACTION, PLATFORMER","July 22, 2009","July 22, 2009",NaN,21.5 Hours Ago
5,19,.hack//G.U. Vol. 1: Rebirth,20.5,26.5,38.0,24.5,80.0,CYBERCONNECT2,BANDAI,"PC, PLAYSTATION 2, PLAYSTATION 4","ACTION, ROLE-PLAYING","October 24, 2006",NaN,"May 18, 2006",20.5 Hours Ago
6,20,.hack//G.U. Vol. 2: Reminisce,22.5,37.0,48.0,31.5,81.0,CYBERCONNECT2,BANDAI,"PLAYSTATION 2, PLAYSTATION 4","ACTION, ROLE-PLAYING","May 08, 2007",NaN,"September 28, 2006",13 Hours Ago
7,21,.hack//G.U. Vol. 3: Redemption,23.5,35.5,52.5,30.0,79.0,CYBERCONNECT2,BANDAI,"PLAYSTATION 2, PLAYSTATION 4","ACTION, ROLE-PLAYING","March 25, 2008",NaN,"January 18, 2007",19.5 Hours Ago
8,22,.hack//Infection,18.0,20.0,28.5,20.5,78.0,CYBERCONNECT2,BANDAI S.A.,PLAYSTATION 2,"ACTION, ROLE-PLAYING","February 11, 2003",NaN,NaN,20 Hours Ago
9,23,.hack//Link,29.0,54.0,NaN,41.5,55.0,CYBERCONNECT2,NAMCO BANDAI GAMES,PSP,ROLE-PLAYING,NaN,NaN,"March 04, 2010",2 Months Ago


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48157 entries, 0 to 48156
Data columns (total 15 columns):
Id               48157 non-null int64
Titulo           48157 non-null object
Main Story       15165 non-null float64
Main + Extras    9929 non-null float64
Completionist    11180 non-null float64
All Styles       18386 non-null float64
Rating           19973 non-null float64
Developers       27844 non-null object
Publishers       26637 non-null object
Playable On      16915 non-null object
Genres           27406 non-null object
NA               25709 non-null object
EU               10630 non-null object
JP               4751 non-null object
Updated          48007 non-null object
dtypes: float64(5), int64(1), object(9)
memory usage: 5.5+ MB


## Cuales son los 100 titulos con mejor rating?

In [9]:
top_100 = df.sort_values('Rating', ascending=False)[0:100]

In [10]:
top_100.head()

,Id,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developers,Publishers,Playable On,Genres,NA,EU,JP,Updated
41218,52455,Psychedelica of the Black Butterfly,7.0,NaN,19.5,13.0,97.0,OTOMATE,AKSYS GAMES,PLAYSTATION VITA,VISUAL NOVEL,"April 27, 2018",NaN,NaN,3 Weeks Ago
10447,12078,Mega Man Unlimited,5.0,5.0,10.5,6.0,97.0,MEGAPHILX,NaN,PC,"ACTION, PLATFORM",NaN,NaN,NaN,6 Months Ago
32263,41422,Umineko,42.0,NaN,80.5,73.0,97.0,07TH EXPANSION,MANGAGAMER,PC,ADVENTURE,"July 08, 2016",NaN,NaN,2 Weeks Ago
9828,11245,Wizball,NaN,NaN,8.0,8.0,97.0,SENSIBLE SOFTWARE,OCEAN SOFTWARE,COMMODORE 64,SHOOT 'EM UP,1987,NaN,NaN,4.5 Days Ago
2361,2646,DJMax Technika Tune,5.0,8.5,95.0,19.0,97.0,"PENTAVISION, NEOWIZ MOBILE",PENTAVISION,PLAYSTATION VITA,MUSIC/RHYTHM,"December 04, 2012",NaN,"September 27, 2012",3.5 Months Ago


## Cuales son los 100 peores titulos?

In [11]:
worst_100 = df.sort_values('Rating', ascending=True)[0:100]

In [12]:
worst_100.head()

,Id,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developers,Publishers,Playable On,Genres,NA,EU,JP,Updated
25047,31367,The Great Battle V,NaN,NaN,4.00,4.00,10.0,SUN L,BANPRESTO,NaN,BEAT 'EM UP,NaN,NaN,"December 22, 1995",8.5 Months Ago
29626,37993,Gears of War 2: Flashback Map Pack DLC,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
29627,37994,Gears of War 2: Snowblind Map Pack DLC,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
39752,50738,Butterfly Moment,NaN,NaN,0.08,0.08,10.0,DAVID MULDER,RED SPLAT GAMES,NaN,VIRTUAL REALITY,"September 22, 2017","September 22, 2017",NaN,3.5 Months Ago
39801,50790,O'Fox Life,NaN,NaN,NaN,NaN,10.0,O'STINKY LAB.,O'STINKY LAB.,NaN,PLATFORM,"October 31, 2017","October 31, 2017",NaN,9 Months Ago


# Modelado

El objetivo de este trabajo es poder crear un modelo predictivo, que a partir de la informacion de un juego, como cual fue su developer, publisher, genero, etc; pueda determinar un rating entre 0 y 100.<br>
Nos interesa especialmente que el modelo pueda acomodarse a valores faltantes en algunos de los campos, para lo cual vamos a utilizar XGBoost.<br>
Al mismo tiempo, vamos a plantear otro modelos de regresion para datasets completos y comparar la performance de ambos.

Para poder proveerle la data al modelo, necesitamos que las columnas que tienen mas de un valor (como Genres) sean transformadas en dummies. <br>
Para eso vamos a definir una funcion que itere sobre las columnas correspondientes, y extraiga todos los valores unicos, para luego crear una columna por cada uno de esos valores y completarla con 1 o 0 dependiendo si pertenece a esa clase o no.

## Defino una funcion para separar los atributos multiclase

In [16]:
def splitter(dataframe, columna):
    #Import de itertools para chainear los valores
    import itertools
    #Genero lista con los valores unicos y lista vacia para ir llenando
    valores_unicos = dataframe[columna].unique()
    valores_individuales = []
    for x in valores_unicos:
        if type(x) == str:
            valores_individuales.append(x.split(','))
        else:
            pass
    valores_individuales = list(itertools.chain.from_iterable(valores_individuales))
    
    #Elimino los espacios al principio y me quedo con los valores unicos
    atributo = []
    for i in range(len(valores_individuales)):
        att = valores_individuales[i].lstrip()
        atributo.append(att)
    valores_individuales = set(atributo)
    
    #Genero un diccionario con llave cada valor unico, y valor la lista de unos y ceros
    diccionario = {}

    for z in valores_individuales:
        lista = []
        for y in range(len(dataframe)):
            try:
                if dataframe[columna][y].find(z) != -1:
                    lista.append(1)
                else:
                    lista.append(0)
            except:
                lista.append(0)
        diccionario[z] = lista
    return diccionario

In [19]:
splitter(df,'Playable On')

{'': [1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
 

### Generos

# ESTA ES LA QUE FUNCA 

In [93]:
#Lista de Generos
import itertools
unique_genres = df['Genres'].unique()
individual_genres = []
for genre in unique_genres:
    if type(genre) == str:
        individual_genres.append(genre.split(','))
    else:
        pass

individual_genres = list(itertools.chain.from_iterable(individual_genres))

#Elimino los espacios al principio y me quedo con los unicos
generos = []
for i in range(len(individual_genres)):
    gen = individual_genres[i].lstrip()
    generos.append(gen)
individual_genres = set(generos)

%%time
#Tengo que loopear sobre individual_genres, generar una columna para cada genero y appendar un 1 o 0 
#si la columna del df tiene ese dato
dict_generos = {}

for genre in individual_genres:
    lista = []
    for i in range(len(df)):
        try:
            if df['Genres'][i].find(genre) != -1:
                lista.append(1)
            else:
                lista.append(0)
        except:
            lista.append(0)
    dict_generos[genre] = lista

# HAY QUE LEVANTAR EXCEPCION PARA LOS NULL

In [80]:
#Lista de Generos
import itertools
unique_genres = df['Genres'].unique()
individual_genres = []
for genre in unique_genres:
    if type(genre) == str:
        individual_genres.append(genre.split(','))
    else:
        pass

individual_genres = list(itertools.chain.from_iterable(individual_genres))
individual_genres = set(individual_genres)

## XGBoost

In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

In [9]:
#Seteamos el indice 
df = df.set_index(df['Id'])
df = df.drop(columns='Id')

In [8]:
#Elimino las observaciones que no tienen completa la variable objetivo
df = df.dropna(subset = ['Rating'])

In [31]:
#Separo la matriz de features y la variable objetivo
X =  df.drop(['Titulo','Rating','Developers','Publishers','NA','EU','JP','Updated'], axis = 1)
y = df['Rating']

In [32]:
#Separo en Train y Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [33]:
#Instancio el Clasificador de XGBoost
xg_reg = xgb.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='reg:linear',\
                           booster='gbtree', n_jobs=-1)

In [34]:
%%time
#Fiteo el modelo
xg_reg.fit(X_train,y_train)

#Predicciones
preds = xg_reg.predict(X_test)

Wall time: 2.89 s


In [35]:
#Score
xg_reg.score(X_test, y_test)

0.19769199453796382

### Como podemos mejorar el score?

# XGBoost + GridSearch

In [47]:
# Definimos una lista de hiperparametros a tunear
l2_reg_range = [3,5,7,10]
profundidad = [2,4,6,8,10]
learn_rate = [0.01,0.03,0.1]

In [48]:
#Generamos la grilla para GridSearch
param_grid = dict(reg_lambda = l2_reg_range, max_depth=profundidad, learning_rate = learn_rate)
print(param_grid)

{'reg_lambda': [3, 5, 7, 10], 'max_depth': [2, 4, 6, 8, 10], 'learning_rate': [0.01, 0.03, 0.1]}


In [49]:
%%time
# Instanciamos el modelo de XGBoost con los parametros de GridSearch y un cross validation de 5 folds
xgb_regressor = xgb.XGBRegressor(n_estimators=100, silent=True, objective='reg:linear',booster='gbtree', n_jobs=-1)
xgb_gs = GridSearchCV(xgb_regressor, param_grid, cv = 5)
xgb_gs.fit(X_train, y_train)

Wall time: 20min


In [50]:
#Vemos los parametros del mejor modelo
xgb_gs.best_estimator_, xgb_gs.best_score_, xgb_gs.best_params_

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
        max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
        n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
        reg_alpha=0, reg_lambda=10, scale_pos_weight=1, seed=None,
        silent=True, subsample=1),
 0.20413076439456873,
 {'learning_rate': 0.1, 'max_depth': 6, 'reg_lambda': 10})

## Utilizamos el mejor modelo para entrenar

In [51]:
xgb_best = xgb.XGBRegressor(n_estimators=100, silent=True, objective='reg:linear',booster='gbtree', n_jobs=-1, \
                           learning_rate = 0.1, max_depth=6,reg_lambda=10)
xgb_best.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=10, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [53]:
xgb_best.score(X_test, y_test)

0.20613014421766385

## CatBoost

In [36]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [41]:
#Separo la matriz de features y la variable objetivo
X_cb =  df.drop(['Titulo','Rating','Developers','Publishers','NA','EU','JP','Updated'], axis = 1)
y_cb = df['Rating'].astype(int)

In [43]:
#Separo en Train y Test
X_train_cb, X_test_cb, y_train_cb, y_test_cb = train_test_split(X_cb, y_cb, test_size=0.25, random_state=1)

In [44]:
cb_model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=3, l2_leaf_reg=7, loss_function='RMSE')

In [45]:
cb_model.fit(X_train_cb, y_train_cb)

0:	learn: 59.5222746	total: 16.1ms	remaining: 1.59s
1:	learn: 54.1467145	total: 28.4ms	remaining: 1.39s
2:	learn: 49.3646450	total: 41.8ms	remaining: 1.35s
3:	learn: 45.1198757	total: 53.8ms	remaining: 1.29s
4:	learn: 41.3022939	total: 67.9ms	remaining: 1.29s
5:	learn: 37.9101091	total: 82.4ms	remaining: 1.29s
6:	learn: 34.9292373	total: 95ms	remaining: 1.26s
7:	learn: 32.3601524	total: 107ms	remaining: 1.23s
8:	learn: 30.0935479	total: 119ms	remaining: 1.21s
9:	learn: 28.0583158	total: 134ms	remaining: 1.2s
10:	learn: 26.3177230	total: 147ms	remaining: 1.19s
11:	learn: 24.7952239	total: 163ms	remaining: 1.2s
12:	learn: 23.5436691	total: 175ms	remaining: 1.17s
13:	learn: 22.4059498	total: 188ms	remaining: 1.16s
14:	learn: 21.4457824	total: 201ms	remaining: 1.14s
15:	learn: 20.6217964	total: 217ms	remaining: 1.14s
16:	learn: 19.9150810	total: 234ms	remaining: 1.14s
17:	learn: 19.3365551	total: 249ms	remaining: 1.13s
18:	learn: 18.8524540	total: 263ms	remaining: 1.12s
19:	learn: 18.44848

In [46]:
#Predecimos sobre el set de testeo
cb_preds = cb_model.predict(X_test_cb, y_test_cb)

TypeError: cannot convert the series to <class 'int'>